In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca
import folium
import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

from calitp import get_engine

engine = get_engine()
connection = engine.connect()

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Join Trips per Stop, Ridership, and ACS data

In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [3]:
# start with trips per stop and ridership
stoptrips = gpd.read_parquet(f"{GCS_FILE_PATH}tbl1_trips_perstop.parquet")
stoptrips >> head (5)

,stop_desc,location_type,route_type,stop_name,stop_id,stop_code,calitp_itp_id,geometry,n_trips_ampeak,n_trips_midday,n_trips_pmpeak,n_trips_sat,n_trips_sun
0,None,None,3,Sunset / Park,8030,8030,182,POINT (-118.26181 34.07778),22.0,30.0,23.0,61.0,61.0
1,None,None,3,Fallbrook / Roscoe,10418,10418,182,POINT (-118.62363 34.21911),15.0,15.0,16.0,25.0,25.0
2,None,None,3,Vanowen / Woodman,15246,15246,182,POINT (-118.43104 34.19407),16.0,15.0,15.0,25.0,25.0
3,None,None,3,Wilshire / Doheny,8417,8417,182,POINT (-118.38985 34.06694),16.0,21.0,16.0,47.0,47.0
4,None,None,3,Fairfax / Rosewood,11986,11986,182,POINT (-118.36159 34.07991),23.0,31.0,27.0,63.0,63.0


In [4]:
ridership = gpd.read_parquet(f"{GCS_FILE_PATH}rider_cleaned_182_2022_03.parquet")
ridership >> head (5)

,calitp_itp_id,stop_id,geometry,sat_ons,sun_ons,weekday_ons
0,182,839,POINT (-118.56218 34.23576),8.0,4.0,61.0
1,182,2786,POINT (-118.36593 33.98405),0.0,0.0,19.0
2,182,6374,POINT (-118.35400 34.24697),0.0,0.0,0.0
3,182,5909,POINT (-118.36770 34.06108),47.0,62.0,855.0
4,182,4511,POINT (-118.15607 33.92645),0.0,0.0,120.0


In [7]:
# join together, keep buses, create total trips per weekday
trips_ridership_joined = (stoptrips
                          >> full_join(_,ridership)
                          >> filter(_.route_type=="3")
                          >> mutate(n_trips_weekday = _.n_trips_ampeak+_.n_trips_midday+_.n_trips_pmpeak)
                         )

trips_ridership_joined >> head (5)

,stop_desc,location_type,route_type,stop_name,stop_id,stop_code,calitp_itp_id,geometry,n_trips_ampeak,n_trips_midday,n_trips_pmpeak,n_trips_sat,n_trips_sun,sat_ons,sun_ons,weekday_ons,n_trips_weekday
0,None,None,3,Sunset / Park,8030,8030,182,POINT (-118.26181 34.07778),22.0,30.0,23.0,61.0,61.0,149.0,135.0,981.0,75.0
1,None,None,3,Fallbrook / Roscoe,10418,10418,182,POINT (-118.62363 34.21911),15.0,15.0,16.0,25.0,25.0,5.0,6.0,94.0,46.0
2,None,None,3,Vanowen / Woodman,15246,15246,182,POINT (-118.43104 34.19407),16.0,15.0,15.0,25.0,25.0,176.0,118.0,1790.0,46.0
3,None,None,3,Wilshire / Doheny,8417,8417,182,POINT (-118.38985 34.06694),16.0,21.0,16.0,47.0,47.0,55.0,33.0,991.0,53.0
4,None,None,3,Fairfax / Rosewood,11986,11986,182,POINT (-118.36159 34.07991),23.0,31.0,27.0,63.0,63.0,34.0,54.0,1175.0,81.0


In [8]:
# project
trips_ridership_joined = trips_ridership_joined.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

# keep point geometry in intermediate table
stop_geom = (trips_ridership_joined
             >> select(_.calitp_itp_id,_.stop_id,_.geometry)
            )

stop_geom.head(5)

,calitp_itp_id,stop_id,geometry
0,182,8030,POINT (160415.334 -436068.849)
1,182,10418,POINT (126798.776 -420939.236)
2,182,15246,POINT (144585.430 -423441.383)
3,182,8417,POINT (148619.650 -437479.512)
4,182,11986,POINT (151203.451 -435996.473)


In [9]:
# add buffers to stops
# this replaces our point geometry with polygons
trips_ridership_joined.geometry = trips_ridership_joined.buffer(402.336)

In [10]:
# ACS data
acs_la = gpd.read_parquet(f"{GCS_FILE_PATH}acs_tbl_lacounty.parquet")
acs_la >> head (5)

,ALAND,geometry,geo_id,total_pop,households,not_us_citizen_pop,black_pop,hispanic_pop,inc_extremelylow,inc_verylow,inc_low,pop_determined_poverty_status,poverty,no_car,no_cars
0,3837562,"POLYGON ((-118.58119 34.14318, -118.58099 34.1...",06037137504,2073.0,694.0,23.0,19.0,64.0,30.0,29.0,102.0,2073.0,90.0,12.0,10.0
1,4472196,"POLYGON ((-118.60573 34.14585, -118.60561 34.1...",06037138000,4673.0,1784.0,198.0,325.0,393.0,270.0,124.0,196.0,4673.0,386.0,0.0,19.0
2,1152031,"POLYGON ((-118.53082 34.18024, -118.52952 34.1...",06037139200,5840.0,2172.0,815.0,153.0,1330.0,242.0,315.0,548.0,5840.0,602.0,66.0,108.0
3,957093,"POLYGON ((-118.37899 34.15409, -118.37888 34.1...",06037143200,4210.0,2083.0,167.0,375.0,539.0,303.0,386.0,378.0,4210.0,455.0,172.0,103.0
4,1649223,"POLYGON ((-118.39648 34.15761, -118.39538 34.1...",06037143300,6730.0,3113.0,421.0,477.0,1270.0,436.0,565.0,415.0,6711.0,384.0,62.0,117.0


In [11]:
# join to job data

jobdata=(pd.read_parquet("gs://calitp-analytics-data/data-analyses/ahsc_grant/job_density")
         >> select(_.geo_id,_.jobs_total)
        )

acs_la = (acs_la
          >> left_join(_,jobdata)
         )

In [12]:
# project
acs_la = acs_la.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [13]:
# join to bus buffers - output is stop-buffer level
# each stop is associated with multiple tracts and a tract can be associated with multiple stops
stops_acs_joined = trips_ridership_joined.sjoin(acs_la, how='inner', predicate='intersects')
stops_acs_joined.geo_id.value_counts()

06037207710    145
06037207900    123
06037207301    122
06037532400    115
06037226002    105
              ... 
06037577000      1
06037267902      1
06037430400      1
06037554516      1
06037264103      1
Name: geo_id, Length: 1741, dtype: int64

In [14]:
# roll back up to stop level - sum the counts of people/households in tracts touching the buffer
stops_acs_rollup = (stops_acs_joined
                    >> group_by(_.calitp_itp_id,_.stop_id, _.n_trips_ampeak,_.n_trips_midday,_.n_trips_pmpeak,
                             _.n_trips_sat,_.n_trips_sun,_.sat_ons,_.sun_ons,_.weekday_ons,_.n_trips_weekday)
                    >> summarize(sum_tracts = _.geo_id.count(),
                                 sum_total_pop = _.total_pop.sum(),
                                 sum_households = _.households.sum(),
                                 sum_not_us_citizen_pop = _.not_us_citizen_pop.sum(),
                                 sum_black_pop = _.black_pop.sum(),
                                 sum_hispanic_pop = _.hispanic_pop.sum(),
                                 sum_inc_extremelylow = _.inc_extremelylow.sum(),
                                 sum_inc_verylow = _.inc_verylow.sum(),
                                 sum_inc_low = _.inc_low.sum(),
                                 sum_pop_determined_poverty_status = _.pop_determined_poverty_status.sum(), #denominator for poverty rate
                                 sum_poverty = _.poverty.sum(),
                                 sum_no_car = _.no_car.sum(), #workers without access to car
                                 sum_no_cars = _.no_cars.sum(), #households without car
                                 sum_land_area = _.ALAND.sum(),
                                 sum_jobs=_.jobs_total.sum()
                                )
                    >> ungroup()
                   )

stops_acs_rollup >> head (5)                    

,calitp_itp_id,stop_id,n_trips_ampeak,n_trips_midday,n_trips_pmpeak,n_trips_sat,n_trips_sun,sat_ons,sun_ons,weekday_ons,n_trips_weekday,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_black_pop,sum_hispanic_pop,sum_inc_extremelylow,sum_inc_verylow,sum_inc_low,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs
0,182,1,13.0,15.0,15.0,44.0,44.0,16.0,24.0,214.0,43.0,1,4031.0,1093.0,696.0,15.0,3766.0,222.0,303.0,197.0,3866.0,338.0,42.0,60.0,3707188,7122.0
1,182,10000002,6.0,7.0,7.0,12.0,12.0,20.0,7.0,102.0,20.0,4,16203.0,4603.0,3061.0,2968.0,9900.0,975.0,1090.0,1146.0,16001.0,2844.0,266.0,364.0,3243477,4193.0
2,182,10033,12.0,15.0,12.0,24.0,24.0,22.0,24.0,205.0,39.0,3,14102.0,4613.0,2455.0,5559.0,7016.0,1357.0,911.0,817.0,13952.0,2738.0,128.0,524.0,2714150,1011.0
3,182,10034,12.0,15.0,12.0,24.0,24.0,7.0,5.0,132.0,39.0,4,18631.0,6567.0,2914.0,8455.0,8467.0,2147.0,1300.0,1201.0,18481.0,3814.0,290.0,945.0,3565703,1809.0
4,182,10035,12.0,15.0,12.0,23.0,23.0,38.0,34.0,371.0,39.0,3,15654.0,5108.0,2848.0,4657.0,9669.0,1324.0,1213.0,1232.0,15469.0,3570.0,207.0,443.0,2568500,1777.0


In [15]:
# derived variables
stops_acs_rollup = (stops_acs_rollup
                     >> mutate(pop_density = _.sum_total_pop/_.sum_land_area,
                               job_density = _.sum_jobs/_.sum_land_area,
                               pct_not_us_citizen_pop = _.sum_not_us_citizen_pop/_.sum_total_pop,
                               pct_black_pop = _.sum_black_pop/_.sum_total_pop,
                               pct_hispanic_pop = _.sum_hispanic_pop/_.sum_total_pop,
                               pct_inc_extremelylow = _.sum_inc_extremelylow/_.sum_households,
                               pct_inc_verylow = _.sum_inc_verylow/_.sum_households,
                               pct_inc_low = _.sum_inc_low/_.sum_households,
                               pct_poverty = _.sum_poverty/_.sum_pop_determined_poverty_status,
                               pct_pop_workers_no_car = _.sum_no_car/_.sum_total_pop,
                               pct_hh_no_cars = _.sum_no_cars/_.sum_households
                              ) 
                    )   

In [16]:
# join back point geometry and save out
stops_acs_rollup_geom = (stop_geom
                         >> left_join(_,stops_acs_rollup)
                        )

type(stops_acs_rollup_geom)

geopandas.geodataframe.GeoDataFrame

In [17]:
shared_utils.utils.geoparquet_gcs_export(stops_acs_rollup_geom, GCS_FILE_PATH, 'analytical_tbl.parquet')

## Correlations with ridership

In [ ]:
# import matplotlib.pyplot as plt

corr = stops_acs_rollup2.corr()
corr.style.background_gradient(cmap='coolwarm', vmin=-1., vmax=1.)

In [ ]:
import seaborn as sns

sns.lmplot(x="pop_density",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Area Population Density")

In [ ]:
sns.lmplot(x="pct_not_us_citizen_pop",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Non-US-Citizen Population")

In [ ]:
sns.lmplot(x="pct_inc_extremelylow",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs Extremely Low Income HH")

In [ ]:
sns.lmplot(x="pct_hh_no_cars",y="weekday_ons", data=stops_acs_rollup2).set(title="Weekday Monthly Ridership vs No-Car HH")